In [1]:
import math
import numpy as np
import pandas as pd
from dateutil.parser import parse

pd.set_option('display.max_columns', None)

In [2]:
def loadCSVFile(csvPath, sepStyle, encodeStyle):
    return pd.read_csv(csvPath, sep=sepStyle, encoding=encodeStyle, low_memory=False)

In [3]:
def mergeDataFrames(leftDF, rightDF, keyColumn, suffixTuple):
    return leftDF.merge(rightDF, how='left', on=keyColumn, suffixes=suffixTuple)

In [4]:
def mergeDtHrUTI(listDtInUTI, listHrInUTI, listDtOutUTI, listHrOutUTI):
    listDTHR_InUTI, listDTHR_OutUTI= [], []
    
    for tplDtHrInUTI in zip(listDtInUTI, listHrInUTI):
        inDtUTI = tplDtHrInUTI[0]
        inHrUTI = tplDtHrInUTI[1]
        if type(inDtUTI) != str and type(inHrUTI) != str:
            if math.isnan(inDtUTI) == True or math.isnan(inHrUTI) == True:
                listDTHR_InUTI.append(np.nan)
        else:
            listDTHR_InUTI.append(' '.join(tplDtHrInUTI))
            
    for tplDtHrOutUTI in zip(listDtOutUTI, listHrOutUTI):
        outDtUTI = tplDtHrOutUTI[0]
        outHrUTI = tplDtHrOutUTI[1]
        if type(outDtUTI) != str and type(outHrUTI) != str:
            if math.isnan(outDtUTI) == True or math.isnan(outHrUTI) == True:
                listDTHR_OutUTI.append(np.nan)
        else:
            listDTHR_OutUTI.append(' '.join(tplDtHrOutUTI))
            
    return listDTHR_InUTI, listDTHR_OutUTI

In [5]:
def timeCounterBtwnDates(bDate, eDate):
    _bDate = parse(bDate, dayfirst=True).ctime()
    _bDate = parse(_bDate, dayfirst=True)
    
    _eDate = parse(eDate, dayfirst=True).ctime()
    _eDate = parse(_eDate, dayfirst=True)

    delta = _eDate - _bDate

    totalDays = delta.total_seconds()/86400
    
    return totalDays

In [14]:
def getDaysCountByRow(row, posREG, posALTA, posInUTI, posOutUTI):
    if type(row[posREG]) == str and type(row[posInUTI]) == str and type(row[posOutUTI]) == str:
        Pre_UTITime = timeCounterBtwnDates(bDate=row[posREG], eDate=row[posInUTI])
        UTI_TIME = timeCounterBtwnDates(bDate=row[posInUTI], eDate=row[posOutUTI])
    
    if type(row[posREG]) == str and type(row[posALTA]) == str and type(row[posInUTI]) == float and type(row[posOutUTI]) == float:
        Pre_UTITime = timeCounterBtwnDates(bDate=row[posREG], eDate=row[posALTA])
        UTI_TIME = 0.0
        
    if type(row[posREG]) == float or type(row[posALTA]) == float:
        Pre_UTITime = np.nan
        UTI_TIME = np.nan
    
    if type(row[posREG]) == float and type(row[posALTA]) == str and type(row[posInUTI]) == float and type(row[posOutUTI]) == float:
        Pre_UTITime = np.nan
        UTI_TIME = np.nan
    
    return Pre_UTITime, UTI_TIME

In [11]:
def getTimeForEachDFRow(DFRows):
    list_PreUTITime, list_UTITime = [], []
    
    for row in DFRows:
        pUTI, UTI = getDaysCountByRow(row=row, posREG=1, posALTA=27, posInUTI=119, posOutUTI=120)
        list_PreUTITime.append(pUTI)
        list_UTITime.append(UTI)
    
    return list_PreUTITime, list_UTITime

In [15]:
if __name__ == '__main__':
    
    # Carrega os dados
    DF_REGISTRO = loadCSVFile(csvPath='atendimentos.csv', encodeStyle='latin1', sepStyle=';')
    DF_ESTUDO = loadCSVFile(csvPath='estudo.csv', encodeStyle='latin1', sepStyle=';')
    DF_UTI = loadCSVFile(csvPath='uti.csv', encodeStyle='latin1', sepStyle=';')
    
    
    #Remove duplicações
    DF_REGISTRO = DF_REGISTRO.drop_duplicates()
    DF_ESTUDO = DF_ESTUDO.drop_duplicates()
    DF_UTI = DF_UTI.drop_duplicates()
    
    
    #União das Tabelas REGISTRO e ESTUDO
    DF_REG_EST = mergeDataFrames(leftDF=DF_REGISTRO,
                                 rightDF=DF_ESTUDO,
                                 keyColumn='CD_PACIENTE',
                                 suffixTuple=('_FROM_REG', '_FROM_EST'))
    
    DF_REG_EST_UTI = mergeDataFrames(leftDF=DF_REG_EST,
                                     rightDF=DF_UTI,
                                     keyColumn='CD_ATENDIMENTO',
                                     suffixTuple=('_FROM_MRG_REG_EST', '_FROM_UTI'))
    
    
    #Remove duplicações
    DF_REG_EST_UTI = DF_REG_EST_UTI.drop_duplicates()
    
    
    #União dos campos DT_ENTRADA + HR_ENTRADA e DT_SAIDA + HR_SAIDA
    lDtInUTI = DF_REG_EST_UTI['DT_ENTRADA'].values.tolist()
    lHrInUTI = DF_REG_EST_UTI['HR_ENTRADA'].values.tolist()
    lDtOutUTI = DF_REG_EST_UTI['DT_SAIDA'].values.tolist()
    lHrOutUTI = DF_REG_EST_UTI['HR_SAIDA'].values.tolist()
    
    listDTHR_InUTI, listDTHR_OutUTI = mergeDtHrUTI(listDtInUTI=lDtInUTI,
                                                   listHrInUTI=lHrInUTI,
                                                   listDtOutUTI=lDtOutUTI,
                                                   listHrOutUTI=lHrOutUTI)
    
    #Adicionando União anterior a tabela geral
    DF_REG_EST_UTI['DTHR_IN_UTI'] = listDTHR_InUTI
    DF_REG_EST_UTI['DTHR_OUT_UTI'] = listDTHR_OutUTI
    
    
    #Calculando e Adicionando numero de dias ante e durante a UTI
    list_PreUTITime, list_UTITime = getTimeForEachDFRow(DFRows=DF_REG_EST_UTI.values.tolist())
    
    DF_REG_EST_UTI['DIAS_PRE_UTI'] = list_PreUTITime
    DF_REG_EST_UTI['DIAS_NA_UTI'] = list_UTITime

In [16]:
DF_REG_EST_UTI

,ID,DTHR_REGISTRO,PORTA_ENTRADA,DATA_INICIO_SINTOMAS,HISTORICO_VIAGEM,PAIS_VISITADO,CIDADES_VISITADAS,DATA_RETORNO,COLETOU_AMOSTRA,LOCAL_COLETA,DATA_COLETA,PCR_INFLUENZA,IFI_OUTRO_VIRUS,PCR_COVID_19,STATUS_COVID_FROM_MRG_REG_EST,STATUS_COVID_ORDEM,DESFECHO,CD_ATENDIMENTO,DS_TP_ATENDIMENTO,CD_PACIENTE,GENERO,IDADE,SN_OBITO,DIAGNOSTICO_SARS,OBITO_SARS,TIPO_SAIDA,TIPO_OBITO,DTHR_ALTA,CD_CID,DS_ESPECIALID_SAI,REINTER_30D,CARATER_ATENDIMENTO,EMERG_SERVICO,EMERG_RISCO,CID_DIAG_ATEND,CID_CAPITULO_DIAG,BLOCO_CENTRAL_ST,HEMODINAMICA_ST,EX_LAB_ST,EX_IMG_ST,EX_OUTROS_ST,COMPETENCIA,PORTA_ENTRADA_FROM_ESTUDO,DATA_INICIO_SINTOMAS_FROM_ESTUDO,HISTORICO_VIAGEM_FROM_ESTUDO,CIDADES_VISITADAS_FROM_ESTUDO,LOCAL_COLETA_FROM_ESTUDO,DATA_COLETA_FROM_ESTUDO,PCR_COVID_19_FROM_ESTUDO,DESFECHO_FROM_ESTUDO,TIPO_REGISTRO,SN_IFI_OV_SINCICIAL_RESP,SN_IFI_OV_PARAINFLU3,SN_IFI_OV_PARAINFLU2,SN_IFI_OV_PARAINFLU1,SN_IFI_OV_INFLUA,SN_IFI_OV_INFLUB,SN_IFI_OV_ADENOVIRUS,SN_CRIT_CONTATO_CASO_COVID,SN_CRIT_CONTATO_CASO_SUSPEITO,SN_CRIT_PROF_SAUDE_HOSPITAL,SN_CRIT_PROF_SAUDE_OUT,SN_CRIT_SRAG,SN_CRIT_SINDROME_GRIPAL,SN_CRIT_IGNORADO,SN_CRIT_OUTROS,SN_CRIT_PROT_COVID19,ESTUDO_TP_EXPOSICAO,ESTUDO_COMORB_PREV_HAS,ESTUDO_COMORB_PREV_IAM,ESTUDO_COMORB_PREV_IC,ESTUDO_COMORB_PREV_DRC,ESTUDO_COMORB_PREV_DMI,ESTUDO_COMORB_PREV_DMII,ESTUDO_COMORB_PREV_CARCINOMA,ESTUDO_COMORB_PREV_ADENOMA,ESTUDO_COMORB_PREV_LINFOMA,ESTUDO_COMORB_PREV_AVC,ESTUDO_COMORB_PREV_ASMA,ESTUDO_COMORB_PREV_DPOC,ESTUDO_COMORB_PREV_OBESIDADE,ESTUDO_COMORB_PREV_EPILEPSIA,ESTUDO_COMORB_PREV_HIGIDO,ESTUDO_COMORB_PREV_OUTRAS,ESTUDO_CHARLSON,ESTUDO_FR_DIA_COLETA,ESTUDO_FC_DIA_COLETA,ESTUDO_PA_DIA_COLETA,ESTUDO_TEMPERATURA_DIA_COLETA,ESTUDO_SATURACAO_DIA_COLETA,ESTUDO_APACHE2,ESTUDO_RISCO,ESTUDO_TRATAMENTO,ESTUDO_COMORB_PREV_TX_ORG_SOL,ESTUDO_CONDUTA,ESTUDO_LOCAL_CONTAMINACAO,ESTUDO_LOCAL_CONTAM_UI,ESTUDO_DT_DESFECHO,ESTUDO_COMORB_PREV_DESCON,ESTUDO_COMORB_PREV_DVP,ESTUDO_COMORB_PREV_DEMENCIA,ESTUDO_COMORB_PREV_DTC,ESTUDO_COMORB_PREV_UP,ESTUDO_COMORB_PREV_DFL,ESTUDO_COMORB_PREV_DFM,ESTUDO_COMORB_PREV_HEMIPLEGIA,ESTUDO_COMORB_PREV_LEUCEMIA,ESTUDO_COMORB_PREV_HIV,ESTUDO_COMORB_PREV_TUL,ESTUDO_COMORB_PREV_TUM,STATUS_COVID_FROM_UTI,CD_PACIENTE_FROM_UTI,CD_LEITO,CD_UNIDADE,DS_UNIDADE,DT_ENTRADA,HR_ENTRADA,DT_SAIDA,HR_SAIDA,DTHR_IN_UTI,DTHR_OUT_UTI,DIAS_PRE_UTI,DIAS_NA_UTI
0,26200,11/03/2021 16:00:00,NaN,NaN,NaN,NaN,NaN,NaN,SIM,PRONTOPUC ADULTO,11/03/2021 16:28:16,NaN,NaN,NEGATIVO,NEGATIVOS,3,NÃO INFORMADO,1589566.0,URGENCIA,1824508.0,MASCULINO,"27,6929223744292",N,NÃO,NÃO,AMBULATORIAL,NaN,11/03/2021 19:21:00,R103,CLINICA MEDICA,NÃO,URGÊNCIA,PRONTOSUS,AMARELO,R103,"Sintomas, sinais e achados anormais de exames ...",NÃO,NÃO,SIM,SIM,NÃO,11/03/2021,NaN,NaN,NaN,NaN,PRONTOPUC ADULTO,11/03/2021 16:28:16,NEGATIVO,NaN,A,N,N,N,N,N,N,N,N,N,NaN,N,N,N,N,N,N,NaN,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,L,NaN,NaN,NaN,N,N,N,N,N,N,N,N,N,N,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.139583,0.000000
1,25919,06/03/2021 09:09:20,NaN,NaN,NaN,NaN,NaN,NaN,SIM,LABORATORIO CLINICO,06/03/2021 09:13:58,NaN,NaN,NEGATIVO,NEGATIVOS,3,NÃO INFORMADO,1584147.0,EXTERNO,1825115.0,FEMININO,"40,4545662100457",N,NÃO,NÃO,AMBULATORIAL,NaN,NaN,NaN,ACUPUNTURA,NÃO,ELETIVO,NÃO INFORMADO,NÃO INFORMADO,B349,Infecciosas e parasitárias,NÃO,NÃO,SIM,NÃO,NÃO,06/03/2021,NaN,NaN,NaN,NaN,LABORATORIO CLINICO,06/03/2021 09:13:58,NEGATIVO,NaN,A,N,N,N,N,N,N,N,N,N,NaN,N,N,N,N,N,N,NaN,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,L,NaN,NaN,NaN,N,N,N,N,N,N,N,N,N,N,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,25975,08/03/2021 08:11:00,NaN,NaN,NaN,NaN,NaN,NaN,SIM,PRONTOPUC ADULTO/INTE,08/03/2021 08:28:39,NaN,NaN,NEGATIVO,NEGATIVOS,3,NÃO INFORMADO,1584472.0,INTERNADO,1832839.0,MASCULINO,"58,8792237442922",S,NÃO,NÃO,ÓBITO,NaN,21/03/2021 18:59:44,A310,CIRURGIA GERAL,NÃO,URGÊNCIA,PRONTOSUS,LARANJA,A419,Infecciosas e parasitárias,SIM,NÃO,SIM,SIM,SIM,08/03/2021,NaN,NaN,NaN,NaN,PRONTOPUC ADULTO/INTE,08/03/2021 0

In [9]:
getDaysCountByRow(DF_REG_EST_UTI.values.tolist()[7], posREG=1, posALTA=27, posInUTI=119, posOutUTI=120)

(10.922141203703704, 0.0)

In [76]:
print('REG', DF_REG_EST_UTI.values.tolist()[7][1])

REG 02/03/2021 15:11:00


In [77]:
print('ALTA', DF_REG_EST_UTI.values.tolist()[7][27])

ALTA 13/03/2021 13:18:53


In [78]:
print('InUTI', DF_REG_EST_UTI.values.tolist()[7][119])

InUTI nan


In [79]:
print('OutUTI', DF_REG_EST_UTI.values.tolist()[7][120])

OutUTI nan


In [58]:
DF_REG_EST_UTI.loc[DF_REG_EST_UTI['CD_ATENDIMENTO']==1584472.0]

,ID,DTHR_REGISTRO,PORTA_ENTRADA,DATA_INICIO_SINTOMAS,HISTORICO_VIAGEM,PAIS_VISITADO,CIDADES_VISITADAS,DATA_RETORNO,COLETOU_AMOSTRA,LOCAL_COLETA,DATA_COLETA,PCR_INFLUENZA,IFI_OUTRO_VIRUS,PCR_COVID_19,STATUS_COVID_FROM_MRG_REG_EST,STATUS_COVID_ORDEM,DESFECHO,CD_ATENDIMENTO,DS_TP_ATENDIMENTO,CD_PACIENTE,GENERO,IDADE,SN_OBITO,DIAGNOSTICO_SARS,OBITO_SARS,TIPO_SAIDA,TIPO_OBITO,DTHR_ALTA,CD_CID,DS_ESPECIALID_SAI,REINTER_30D,CARATER_ATENDIMENTO,EMERG_SERVICO,EMERG_RISCO,CID_DIAG_ATEND,CID_CAPITULO_DIAG,BLOCO_CENTRAL_ST,HEMODINAMICA_ST,EX_LAB_ST,EX_IMG_ST,EX_OUTROS_ST,COMPETENCIA,PORTA_ENTRADA_FROM_ESTUDO,DATA_INICIO_SINTOMAS_FROM_ESTUDO,HISTORICO_VIAGEM_FROM_ESTUDO,CIDADES_VISITADAS_FROM_ESTUDO,LOCAL_COLETA_FROM_ESTUDO,DATA_COLETA_FROM_ESTUDO,PCR_COVID_19_FROM_ESTUDO,DESFECHO_FROM_ESTUDO,TIPO_REGISTRO,SN_IFI_OV_SINCICIAL_RESP,SN_IFI_OV_PARAINFLU3,SN_IFI_OV_PARAINFLU2,SN_IFI_OV_PARAINFLU1,SN_IFI_OV_INFLUA,SN_IFI_OV_INFLUB,SN_IFI_OV_ADENOVIRUS,SN_CRIT_CONTATO_CASO_COVID,SN_CRIT_CONTATO_CASO_SUSPEITO,SN_CRIT_PROF_SAUDE_HOSPITAL,SN_CRIT_PROF_SAUDE_OUT,SN_CRIT_SRAG,SN_CRIT_SINDROME_GRIPAL,SN_CRIT_IGNORADO,SN_CRIT_OUTROS,SN_CRIT_PROT_COVID19,ESTUDO_TP_EXPOSICAO,ESTUDO_COMORB_PREV_HAS,ESTUDO_COMORB_PREV_IAM,ESTUDO_COMORB_PREV_IC,ESTUDO_COMORB_PREV_DRC,ESTUDO_COMORB_PREV_DMI,ESTUDO_COMORB_PREV_DMII,ESTUDO_COMORB_PREV_CARCINOMA,ESTUDO_COMORB_PREV_ADENOMA,ESTUDO_COMORB_PREV_LINFOMA,ESTUDO_COMORB_PREV_AVC,ESTUDO_COMORB_PREV_ASMA,ESTUDO_COMORB_PREV_DPOC,ESTUDO_COMORB_PREV_OBESIDADE,ESTUDO_COMORB_PREV_EPILEPSIA,ESTUDO_COMORB_PREV_HIGIDO,ESTUDO_COMORB_PREV_OUTRAS,ESTUDO_CHARLSON,ESTUDO_FR_DIA_COLETA,ESTUDO_FC_DIA_COLETA,ESTUDO_PA_DIA_COLETA,ESTUDO_TEMPERATURA_DIA_COLETA,ESTUDO_SATURACAO_DIA_COLETA,ESTUDO_APACHE2,ESTUDO_RISCO,ESTUDO_TRATAMENTO,ESTUDO_COMORB_PREV_TX_ORG_SOL,ESTUDO_CONDUTA,ESTUDO_LOCAL_CONTAMINACAO,ESTUDO_LOCAL_CONTAM_UI,ESTUDO_DT_DESFECHO,ESTUDO_COMORB_PREV_DESCON,ESTUDO_COMORB_PREV_DVP,ESTUDO_COMORB_PREV_DEMENCIA,ESTUDO_COMORB_PREV_DTC,ESTUDO_COMORB_PREV_UP,ESTUDO_COMORB_PREV_DFL,ESTUDO_COMORB_PREV_DFM,ESTUDO_COMORB_PREV_HEMIPLEGIA,ESTUDO_COMORB_PREV_LEUCEMIA,ESTUDO_COMORB_PREV_HIV,ESTUDO_COMORB_PREV_TUL,ESTUDO_COMORB_PREV_TUM,STATUS_COVID_FROM_UTI,CD_PACIENTE_FROM_UTI,CD_LEITO,CD_UNIDADE,DS_UNIDADE,DT_ENTRADA,HR_ENTRADA,DT_SAIDA,HR_SAIDA,DTHR_IN_UTI,DTHR_OUT_UTI
2,25975,08/03/2021 08:11:00,NaN,NaN,NaN,NaN,NaN,NaN,SIM,PRONTOPUC ADULTO/INTE,08/03/2021 08:28:39,NaN,NaN,NEGATIVO,NEGATIVOS,3,NÃO INFORMADO,1584472.0,INTERNADO,1832839.0,MASCULINO,"58,8792237442922",S,NÃO,NÃO,ÓBITO,NaN,21/03/2021 18:59:44,A310,CIRURGIA GERAL,NÃO,URGÊNCIA,PRONTOSUS,LARANJA,A419,Infecciosas e parasitárias,SIM,NÃO,SIM,SIM,SIM,08/03/2021,NaN,NaN,NaN,NaN,PRONTOPUC ADULTO/INTE,08/03/2021 08:28:39,NEGATIVO,INTERNAÇÃO,A,N,N,N,N,N,N,N,N,N,NaN,N,N,N,N,N,N,NaN,N,N,N,N,N,N,N,N,N,N,N,N,N,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,L,NaN,NaN,NaN,N,N,N,N,N,N,N,N,N,N,N,N,NEGATIVOS,1832839.0,1649.0,8.0,UTI,08/03/2021,13:31:53,18/03/2021,23:22:48,08/03/2021 13:31:53,18/03/2021 23:22:48


In [48]:
getDaysCountByRow(DF_REG_EST_UTI.values.tolist()[2], posREG=1, posALTA=27, posInUTI=119, posOutUTI=120)

(0.22283564814814816, 13.227673611111111)

In [53]:
print('REG', DF_REG_EST_UTI.values.tolist()[2][1])

REG 08/03/2021 08:11:00


In [54]:
print('ALTA', DF_REG_EST_UTI.values.tolist()[2][27])

ALTA 21/03/2021 18:59:44


In [55]:
print('InUTI', DF_REG_EST_UTI.values.tolist()[2][119])

InUTI 08/03/2021 13:31:53


In [56]:
print('OutUTI', DF_REG_EST_UTI.values.tolist()[2][120])

OutUTI 18/03/2021 23:22:48
